### 주의
- 함수 현재 index 500까지 설정되어있습니다. (테스트) 
- `get_station_across` 3번 주석 해제해주세요 (전체)

In [41]:
# 추가되는 패키지는 여기에서 import 해주세요
import warnings
warnings.filterwarnings('ignore')

import missingno
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import geopy.distance
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 500)

import numpy as np
import datetime
import statsmodels.api as sm

In [42]:
# 데이터 load
train = pd.read_csv('train.csv', date_parser='date')
# 분석을 위해 컬럼명 수정
train.rename(columns={'6~7_ride': 'ride6', '7~8_ride': 'ride7', '8~9_ride': 'ride8',\
                      '9~10_ride': 'ride9', '10~11_ride': 'ride10', '11~12_ride': 'ride11', 
                      '6~7_takeoff': 'off6', '7~8_takeoff': 'off7', '8~9_takeoff': 'off8', \
                      '9~10_takeoff': 'off9', '10~11_takeoff': 'off10', '11~12_takeoff': 'off11',\
                      '18~20_ride': 'ride18'},inplace=True)


# 시작 날짜와, 끝 날짜 확인
print(train['date'].min())
print(train['date'].max())
# date 타입 변경
train['date'] = train['date'].astype('datetime64')
# 9월 1일 ~ 9월 30일 데이터이니까, day만 따로 가져옴
train['date_day'] = train['date'].dt.day
# 월(0) ~ 일(6) 데이터로 변환
train['date_dayofweek'] = train['date'].dt.dayofweek
train['date_dayofname']= train['date'].dt.day_name()
train[['date','date_dayofweek','date_day','date_dayofname']]

#평일과 휴일을 나누기, 금토일 =1, 나머지는 0
train.loc[train['date_dayofweek'] == 0, 'holiday' ] = 0
train.loc[train['date_dayofweek'] == 1, 'holiday' ] = 0
train.loc[train['date_dayofweek'] == 2, 'holiday' ] = 0
train.loc[train['date_dayofweek'] == 3, 'holiday' ] = 0
train.loc[train['date_dayofweek'] == 4, 'holiday' ] = 1
train.loc[train['date_dayofweek'] == 5, 'holiday' ] = 1
train.loc[train['date_dayofweek'] == 6, 'holiday' ] = 1
# 추석 추가도 공휴일로 처리
train.loc[train['date'] == '2019-09-14', 'holiday'] = 1
train.loc[train['date'] == '2019-09-13', 'holiday'] = 1
train.loc[train['date'] == '2019-09-12', 'holiday'] = 1


# 시내와 시외를 int형으로 원핫인코딩
train.loc[train['in_out'] == '시외','in_out'] = 1
train.loc[train['in_out'] == '시내','in_out'] = 0

train.tail()

2019-09-01
2019-09-30


,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,ride6,ride7,ride8,ride9,ride10,ride11,off6,off7,off8,off9,off10,off11,ride18,date_day,date_dayofweek,date_dayofname,holiday
415418,415418,2019-09-30,32820000,0,1129,한림환승정류장(한림리),33.41437,126.26336,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30,0,Monday,0.0
415419,415419,2019-09-30,32820000,0,1564,제주시외버스터미널,33.49946,126.51479,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30,0,Monday,0.0
415420,415420,2019-09-30,32820000,0,2322,해병부대,33.23100,126.26273,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,30,0,Monday,0.0
415421,415421,2019-09-30,32820000,0,3291,애월환승정류장(애월리),33.46483,126.31870,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30,0,Monday,0.0
415422,415422,2019-09-30,32820000,0,6115100,서귀포시외버스터미널,33.24873,126.50799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,30,0,Monday,0.0


In [43]:
total = train.groupby(['station_name', 'station_code']).count()
total

id  date  bus_route_id  in_out  latitude  \
station_name station_code                                              
(구)구판장       4211          33   33    33            33      33         
             4212          10   10    10            10      10         
(구)중문동주민센터   2058          263  263   263           263     263        
             2059          299  299   299           299     299        
(구)중앙파출소     1952          280  280   280           280     280        
             2080          177  177   177           177     177        
             2696          76   76    76            76      76         
(구)화산초등학교    2146          6    6     6             6       6          
             4210          18   18    18            18      18         
1100고지휴게소    1245          18   18    18            18      18         
             2638          25   25    25            25      25         
1100도로입구     1653          74   74    74            74      74         
4.3평화공원      1526          44   44    44            44      44         
             1527          10   10    10            10      10         
4.3평화공원입구    1529          14   14    14            14      14         
6호광장         694           162  162   162           162     162        
             695           66   66    66            66      66         
911의원        1243          46   46    46            46      46         
             1244          84   84    84            84      84         
GM빌라         538           61   61    61            61      61         
             539           68   68    68            68      68         
KIS빌리지       4086          5    5     5             5       5          
             4087          18   18    18            18      18         
LH12단지       2767          89   89    89            89      89         
             2768          124  124   124           124     124        
LH아파트        2083          464  464   464           464     464        
L마트          1937          413  413   413           413     413        
NHF삼화1단지     3510          379  379   379           379     379        
NLCS제주       2741          74   74    74            74      74         
             2742          72   72    72            72      72         
...                        ..   ..    ..            ..      ..         
회수마을회관       3073          137  137   137           137     137        
회수사거리        3590          22   22    22            22      22         
             3591          16   16    16            16      16         
회수삼거리        1649          69   69    69            69      69         
             1650          39   39    39            39      39         
회천입구         263           151  151   151           151     151        
효돈농협하나로마트    3389          93   93    93            93      93         
             3390          35   35    35            35      35         
효돈중학교        1817          248  248   248           248     248        
             1818          317  317   317           317     317        
효돈초등학교       1819          357  357   357           357     357        
             2072          259  259   259           259     259        
효돈축구공원       3387          22   22    22            22      22         
             3388          4    4     4             4       4          
효례교          2075          85   85    85            85      85         
효성빌라         1799          61   61    61            61      61         
             1800          53   53    53            53      53         
휴애리자연생활공원    3614          11   11    11            11      11         
             3615          20   20    20            20      20         
흙담솔사가        3478          23   23    23            23      23         
             3479          39   39    39            39      39         
흙통           1922          34   34    34            34      34         
             1923          46   46    46            46      46      

In [44]:
station = {}
for idx in total.index:
    if idx[0] not in station.keys():
        station[idx[0]] = 1
    else:
        station[idx[0]] += 1

df = pd.DataFrame()
df['station_name'] = station.keys()
df['counts'] = station.values()
df = df.sort_values(['counts','station_name'], ascending = [False, True])
df.head()

,station_name,counts
1246,신안동,8
1178,금악리,7
713,비석거리,6
1383,성읍1리,6
1654,산방산,5


In [45]:
total.tail()

id  date  bus_route_id  in_out  latitude  \
station_name station_code                                              
흥덕사          4199          11   11    11            11      11         
             4200          10   10    10            10      10         
희진주유소        2584          74   74    74            74      74         
             2585          102  102   102           102     102        
히귀반석         503           73   73    73            73      73         

                           longitude  ride6  ride7  ride8  ride9  ride10  \
station_name station_code                                                  
흥덕사          4199          11         11     11     11     11     11       
             4200          10         10     10     10     10     10       
희진주유소        2584          74         74     74     74     74     74       
             2585          102        102    102    102    102    102      
히귀반석         503           73         73     73     73     73     73       

                           ride11  off6  off7  off8  off9  off10  off11  \
station_name station_code                                                 
흥덕사          4199          11      11    11    11    11    11     11      
             4200          10      10    10    10    10    10     10      
희진주유소        2584          74      74    74    74    74    74     74      
             2585          102     102   102   102   102   102    102     
히귀반석         503           73      73    73    73    73    73     73      

                           ride18  date_day  date_dayofweek  date_dayofname  \
station_name station_code                                                     
흥덕사          4199          11      11        11              11               
             4200          10      10        10              10               
희진주유소        2584          74      74        74              74               
             2585          102     102       102             102              
히귀반석         503           73      73        73              73               

                           holiday  
station_name station_code           
흥덕사          4199          11       
             4200          10       
희진주유소        2584          74       
             2585          102      
히귀반석         503           73

In [46]:
coords_jejusi = (33.500770, 126.522761) #제주시의 위도 경도
coords_seoquipo = (33.259429, 126.558217) #서귀포시의 위도 경도
train['dis_jejusi'] = [geopy.distance.vincenty((train['latitude'].iloc[i],train['longitude'].iloc[i]), coords_jejusi).km for i in range(len(train))]
train['dis_seoquipo'] = [geopy.distance.vincenty((train['latitude'].iloc[i],train['longitude'].iloc[i]), coords_seoquipo).km for i in range(len(train))]

# 메인

In [99]:
def get_station_across():
    
#     d2 = train
    d2 = train.loc[0:500] ## 데이터 확인용 
    blank = pd.DataFrame()
    
    for i in d2.index:
        print(str(i)+"/"+ str(len(d2)))
        test = d2.loc[i]
        test_date = d2.loc[i, 'date']
        test_st_name = d2.loc[i, 'station_name']
        test_dis = d2.loc[i, 'dis_jejusi']       
        

        right = d2.query('date == @test_date & station_name == @test_st_name').reset_index()
        right['code'] = abs(right['station_code'] - d2.loc[i, 'station_code'])
        right['standard'] = abs(right['dis_jejusi'] - d2.loc[i, 'dis_jejusi'])
        right['route_id'] = abs(right['bus_route_id'] - d2.loc[i, 'bus_route_id'])

        bag = pd.DataFrame(np.zeros(len(right.columns))).T #26
        bag.columns = right.columns
        bag.station_name = right.station_name
        bag.station_code = right.station_code
        bag.date = right.date
        bag.in_out = right.in_out
        bag.bus_route_id = right.bus_route_id

        
        try:
            A = pd.DataFrame(test).T.merge(pd.DataFrame(right.sort_values(['route_id','code','standard']).reset_index(drop = True).loc[1]).T, on = 'station_name')
        except:
            A = pd.DataFrame(test).T.merge(bag, on = 'station_name')
        
        blank = pd.concat([blank, A])
    blank.drop(['code','standard','index','id_y','holiday_y','date_day_y','date_dayofweek_y','date_dayofname_y','dis_jejusi_y','dis_seoquipo_y','holiday_y','dis_jejusi_y','dis_seoquipo_y','route_id','ride18_y'], axis = 1, inplace = True)
    blank.reset_index(inplace = True)
    return blank

In [100]:
new_train = get_station_across()
new_train.to_csv('newtrain.csv')
new_train[['date_x','station_code_x','station_name','date_y','station_code_y']]

0/501
1/501
2/501
3/501
4/501
5/501
6/501
7/501
8/501
9/501
10/501
11/501
12/501
13/501
14/501
15/501
16/501
17/501
18/501
19/501
20/501
21/501
22/501
23/501
24/501
25/501
26/501
27/501
28/501
29/501
30/501
31/501
32/501
33/501
34/501
35/501
36/501
37/501
38/501
39/501
40/501
41/501
42/501
43/501
44/501
45/501
46/501
47/501
48/501
49/501
50/501
51/501
52/501
53/501
54/501
55/501
56/501
57/501
58/501
59/501
60/501
61/501
62/501
63/501
64/501
65/501
66/501
67/501
68/501
69/501
70/501
71/501
72/501
73/501
74/501
75/501
76/501
77/501
78/501
79/501
80/501
81/501
82/501
83/501
84/501
85/501
86/501
87/501
88/501
89/501
90/501
91/501
92/501
93/501
94/501
95/501
96/501
97/501
98/501
99/501
100/501
101/501
102/501
103/501
104/501
105/501
106/501
107/501
108/501
109/501
110/501
111/501
112/501
113/501
114/501
115/501
116/501
117/501
118/501
119/501
120/501
121/501
122/501
123/501
124/501
125/501
126/501
127/501
128/501
129/501
130/501
131/501
132/501
133/501
134/501
135/501
136/501
137/501
138/50

,date_x,station_code_x,station_name,date_y,station_code_y
0,2019-09-01 00:00:00,344,제주썬호텔,2019-09-01 00:00:00,343
1,2019-09-01 00:00:00,357,한라병원,2019-09-01 00:00:00,358
2,2019-09-01 00:00:00,432,정존마을,2019-09-01 00:00:00,431
3,2019-09-01 00:00:00,1579,제주국제공항(600번),2019-09-01 00:00:00,1579
4,2019-09-01 00:00:00,1646,중문관광단지입구,2019-09-01 00:00:00,1645
5,2019-09-01 00:00:00,1648,예래입구,2019-09-01 00:00:00,1647
6,2019-09-01 00:00:00,1660,제주국제컨벤션센터중문대포해안주상절리대,2019-09-01 00:00:00,1660
7,2019-09-01 00:00:00,1666,창천리,2019-09-01 00:00:00,1665
8,2019-09-01 00:00:00,1726,강정농협,2019-09-01 00:00:00,1726
9,2019-09-01 00:00:00,2008,서귀포항,2019-09-01 00:00:00,2105


In [101]:
#확인을 위해 station_code_y와 latitude_y, longitude_y 칼럼을 남겼습니다. OLS 직전에 삭제하시면 될 것 같습니다.
new_train.tail()

,index,id_x,date_x,bus_route_id_x,in_out_x,station_code_x,station_name,latitude_x,longitude_x,ride6_x,ride7_x,ride8_x,ride9_x,ride10_x,ride11_x,off6_x,off7_x,off8_x,off9_x,off10_x,off11_x,ride18_x,date_day_x,date_dayofweek_x,date_dayofname_x,holiday_x,dis_jejusi_x,dis_seoquipo_x,date_y,bus_route_id_y,in_out_y,station_code_y,latitude_y,longitude_y,ride6_y,ride7_y,ride8_y,ride9_y,ride10_y,ride11_y,off6_y,off7_y,off8_y,off9_y,off10_y,off11_y
496,0,496,2019-09-01 00:00:00,21610000,0,478,대림2차아파트,33.479,126.488,0,0,0,0,0,0,1,0,0,0,0,1,0,1,6,Sunday,1,4.03312,25.2143,2019-09-01 00:00:00,21610000,0,478,0,0,0,0,0,0,0,0,0,0,0,0,0,0
497,0,497,2019-09-01 00:00:00,21610000,0,480,연동대림1차아파트,33.4803,126.49,0,0,0,0,0,0,0,0,1,0,0,0,0,1,6,Sunday,1,3.81787,25.31,2019-09-01 00:00:00,21610000,0,481,33.4807,126.49,0,0,0,1,0,0,0,0,0,0,0,0
498,0,498,2019-09-01 00:00:00,21610000,0,481,연동대림1차아파트,33.4807,126.49,0,0,0,1,0,0,0,0,0,0,0,0,1,1,6,Sunday,1,3.7768,25.3508,2019-09-01 00:00:00,21610000,0,480,33.4803,126.49,0,0,0,0,0,0,0,0,1,0,0,0
499,0,499,2019-09-01 00:00:00,21610000,0,520,오름가름,33.4716,126.508,0,0,0,0,0,0,0,0,0,0,1,0,0,1,6,Sunday,1,3.5138,23.9995,2019-09-01 00:00:00,21610000,0,520,0,0,0,0,0,0,0,0,0,0,0,0,0,0
500,0,500,2019-09-01 00:00:00,21610000,0,533,오등동,33.4751,126.529,0,1,1,0,1,0,0,0,0,0,0,0,1,1,6,Sunday,1,2.90909,24.0745,2019-09-01 00:00:00,21610000,0,533,0,0,0,0,0,0,0,0,0,0,0,0,0,0
